In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
data_path = r'C:\Users\roseh\Desktop\NYBG_R\Lamont\data'

### Clean metadata file

In [3]:
# list LDEO metadata sheet names
sheets = ['220610_L','220613_L','220614_L','220617_L','220621_L','220629_L',
          '220707_L','220711_L','220720_L','220803_L','220808_L','220818_L',
          '220902_L','220916_L','221014_L']
# read in excel sheets
metadata = pd.read_excel(os.path.join(data_path,'Metadata_1.xlsx'), sheet_name=sheets,skiprows=3)
# convert dictionary into list of dataframes, adding date column
dataframes = []
for name,group in metadata.items():
    group['date'] = '20' + name[:6]
    dataframes.append(group)
# concatenate dataframes  
metadata = pd.concat(dataframes)  
# select desired columns  
metadata = metadata[['Collar','Chamber Used','Temperature (° C)','Soil Moisture','Air Temp (° C)','Instrument','NDVI','date']]
# convert date column to date data type
metadata['date'] = pd.to_datetime(metadata['date'])
# select only Respiration records
metadata = metadata.loc[metadata['Chamber Used'] != 'NEE']
# edit collar column
metadata['Collar'] = metadata['Collar'].str[:6]

### Clean 7810 files

In [6]:
# read in 7810 csv files
extension = 'csv'
filenames_7810 = [i for i in glob.glob(os.path.join(data_path,'*7810.{}'.format(extension)))]
# concatenate files
combined_7810 = pd.concat([pd.read_csv(f,header=1) for f in filenames_7810 ])
# select columns
combined_7810 = combined_7810[['TA initial_value','FCO2_DRY','FCO2_DRY LIN','LABEL','DATE_TIME initial_value','FCO2_DRY LIN_CV']]
# drop non-data columns
combined_7810 = combined_7810.loc[combined_7810['TA initial_value'] != '[C]']
# rename columns
combined_7810.columns = ['chamber_temp','exp_flux','lin_flux','label','datetime','cv']
# convert datetime column to date data type
combined_7810['datetime'] = pd.to_datetime(combined_7810['datetime'])
# convert other columns to float data type
for col in ['chamber_temp','exp_flux','lin_flux','cv']:
    combined_7810[col] = combined_7810[col].astype('float')
# create column with just the date  
combined_7810['date'] = pd.to_datetime(combined_7810['datetime'].dt.date) 
# create type column
combined_7810['type'] = combined_7810['label'].str[3:5]
# edit lable column
combined_7810['label'] = combined_7810['label'].str[:6]
# perform QC to remove rows with bad date values
combined_7810 = combined_7810.loc[((combined_7810['lin_flux'] - combined_7810['exp_flux']).abs() / combined_7810['lin_flux'] < 0.1) | (combined_7810['cv'] < 1.2)]

### Clean 8100 files

In [7]:
# read in 8100 csv files
extension = 'csv'
filenames_8100 = [i for i in glob.glob(os.path.join(data_path,'*8100.{}'.format(extension)))]
# concatenate files
combined_8100 = pd.concat([pd.read_csv(f,header=1) for f in filenames_8100])
# select columns
combined_8100 = combined_8100[['TA initial_value','FCO2_DRY','FCO2_DRY LIN','COMMENT','DATE initial_value','FCO2_DRY LIN_CV']]
# remove non-data rows
combined_8100 = combined_8100.loc[combined_8100['TA initial_value'] != '[C]']
# rename columns
combined_8100.columns = ['chamber_temp','exp_flux','lin_flux','label','datetime','cv']
# convert datetime column to date data type
combined_8100['datetime'] = pd.to_datetime(combined_8100['datetime'])
# convert other columns to float data type
for col in ['chamber_temp','exp_flux','lin_flux','cv']:
    combined_8100[col] = combined_8100[col].astype('float')
# create column with just the date  
combined_8100['date'] = pd.to_datetime(combined_8100['datetime'].dt.date)
# create type column
combined_8100['type'] = combined_8100['label'].str[3:5]
# edit lable column
combined_8100['label'] = combined_8100['label'].str[:6]
# perform QC to remove rows with bad date values
combined_8100 = combined_8100.loc[((combined_8100['lin_flux'] - combined_8100['exp_flux']).abs() / combined_8100['lin_flux'] < 0.1) | (combined_8100['cv'] < 1.2)]   

### Concatenate and merge cleaned files

In [15]:
# concatenate 7810 and 8100 files
flux = pd.concat([combined_7810,combined_8100])

In [18]:
# merge flux file with metadata file
merge = flux.merge(metadata,left_on=['date','label'],right_on=['date','Collar'],how='outer')

In [20]:
# read merged file to csv
merge.to_csv(os.path.join(data_path,'lamont_all_main.csv'),index=False)